In [4]:
import torch
import torch.nn as nn
import numpy as np

In [5]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [6]:
x = np.array([2.0, 1.0, 0.1])
outputs = softmax(x)
print('softmax numpy: ', outputs)

softmax numpy:  [0.65900114 0.24243297 0.09856589]


In [7]:
x = torch.tensor([2.0, 1.0, 0.1])
outputs = torch.softmax(x, dim=0) # dim=0 computes along the first axis
print(outputs)

tensor([0.6590, 0.2424, 0.0986])


In [8]:
def cross_entropy(actual, predicted):
    loss = -np.sum(actual * np.log(predicted))
    return loss # / float(predicted.shape[0]) - this is normalization

In [13]:
# y must be one hot encoded
# if class 0: [1 0 0]
# if class 1: [0 1 0]
# if class 2: [0 0 1]
Y = np.array([1, 0, 0])

In [14]:
# y_pred has probabilities
Y_pred_good = np.array([0.7, 0.2, 0.1])
Y_pred_bad = np.array([0.1, 0.3, 0.6])
l1 = cross_entropy(Y, Y_pred_good)
l2 = cross_entropy(Y, Y_pred_bad)
print(f'Loss1 numpy: {l1:.4f}')
print(f'Loss1 numpy: {l2:.4f}')

Loss1 numpy: 0.3567
Loss1 numpy: 2.3026


### Torch implementation

In [15]:
# nn.CrossEntropyLoss applies:
# nn.LogSoftmax = nn.LLLoss (negative log likelihood loss)

# -> No Softmax in last layer

# Y has class labels, not One-Hot encoding!
# Y_pred has raw scores (logits), no Softmax!

loss = nn.CrossEntropyLoss()

In [16]:
Y = torch.tensor([0])

# size = n_samples * n_classes = 1x3
Y_pred_good = torch.tensor([[2.0, 1.0, 0.1]]) # raw values not softmax
Y_pred_bad = torch.tensor([[0.5, 2.0, 0.3]])

In [21]:
l1 = loss(Y_pred_good, Y)
l2 = loss(Y_pred_bad, Y)
print(f'Loss1 : {l1:.4f}')
print(f'Loss1 : {l2:.4f}')

Loss1 : 0.4170
Loss1 : 1.8406


In [22]:
_, predictions1 = torch.max(Y_pred_good, 1)
_, predictions2 = torch.max(Y_pred_bad, 1)
print(predictions1)
print(predictions2)

tensor([0])
tensor([1])


In [23]:
# 3 samples
Y = torch.tensor([2, 0, 1])

# size = n_samples * n_classes = 3x3
Y_pred_good = torch.tensor([[0.1, 1.0, 2.1], [2.0, 1.0, 0.1], [1.0, 3.0, 0.1]]) # raw values not softmax
Y_pred_bad = torch.tensor([[2.1, 1.0, 0.1], [1.0, 1.0, 2.1], [0.1, 3.0, 0.1]])

In [24]:
l1 = loss(Y_pred_good, Y)
l2 = loss(Y_pred_bad, Y)
print(f'Loss1 : {l1:.4f}')
print(f'Loss1 : {l2:.4f}')

Loss1 : 0.3251
Loss1 : 1.3662


In [25]:
_, predictions1 = torch.max(Y_pred_good, 1)
_, predictions2 = torch.max(Y_pred_bad, 1)
print(predictions1)
print(predictions2)

tensor([2, 0, 1])
tensor([0, 2, 1])


### Binary Cross Entropy

In [26]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(NeuralNet, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out)
        
        # sigmoid at the end
        y_pred = torch.sigmoid(out)
        return y_pred

In [28]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = NeuralNet(input_size=28*28, hidden_size=5)
model.to(device)

criterion = nn.BCELoss()